Primero vamos a conectarnos con el servidor de origen y listar sus proyectos

In [ ]:
from redminelib import Redmine
import cfg_src_rm

if cfg_src_rm.ignore_cert:
    src_rm = Redmine(cfg_src_rm.server_url,key=cfg_src_rm.api_key, requests={'verify': False})
else:
    src_rm = Redmine(cfg_src_rm.server_url,key=cfg_src_rm.api_key)

src_prjs = src_rm.project.all()

print("Proyectos del servidor origen:")
for p in src_prjs:
    print ("\t",p.identifier," \t| ",p.name)


Ahora vamos a averiguar los identificadores de los campos personalizados que necesitaremos, mientras construimos un diccionario de ellos:

In [ ]:
src_cfs = {}
for cf in src_rm.custom_field.all():
    print(cf)
    src_cfs[cf.name] = cf
    
print(src_cfs.keys())

Listaremos las tareas mientras construimos un diccionario para saber encontrar las tareas por subject

In [ ]:
src_prj = src_rm.project.get(cfg_src_rm.project_id)
print ("Obtenemos proyecto: ",src_prj.identifier," | ",src_prj.name)

src_prj_issues = src_rm.issue.filter(
    project_id=src_prj.id,
    sort='subject:asc'
)

src_tsks = {}
src_tsks_id = {}
for i in src_prj_issues:
    print(i.subject)
    src_tsks[i.subject] = src_rm.issue.get(i.id)
    src_tsks_id[i.id] = src_rm.issue.get(i.id)
    print(i.id)
    
print(src_tsks.keys())

Ahora vamos a conectarnos con el servidor de destino y listar sus proyectos

In [ ]:
from redminelib import Redmine
import cfg_dst_rm

if cfg_dst_rm.ignore_cert:
    dst_rm = Redmine(cfg_dst_rm.server_url,key=cfg_dst_rm.api_key, requests={'verify': False})
else:
    dst_rm = Redmine(cfg_dst_rm.server_url,key=cfg_dst_rm.api_key)

dst_prjs = dst_rm.project.all()

print("Proyectos del servidor destino:")
for p in dst_prjs:
    print ("\t",p.identifier," \t| ",p.name)

Ahora vamos a averiguar los identificadores de los campos personalizados que necesitaremos, mientras construimos un diccionario de ellos:

In [ ]:
dst_cfs = {}
for cf in dst_rm.custom_field.all():
    print(cf)
    dst_cfs[cf.name] = cf
    
print(dst_cfs.keys())

Listaremos las tareas mientras construimos un diccionario para saber encontrar las tareas por subject

In [ ]:
dst_prj = dst_rm.project.get(cfg_dst_rm.project_id)
print ("Obtenemos proyecto: ",dst_prj.identifier," | ",dst_prj.name)

dst_prj_issues = dst_rm.issue.filter(
    project_id=dst_prj.id,
    sort='subject:asc'
)

dst_tsks = {}
dst_tsks_id = {}
for i in dst_prj_issues:
    print(i.subject)
    dst_tsks[i.subject] = dst_rm.issue.get(i.id)
    dst_tsks_id[i.id] = dst_rm.issue.get(i.id)
    
print(dst_tsks.keys())

In [ ]:
print("Source Redmine users")
src_users_login = {}
src_users_id = {}
for mb in src_prj.memberships:
    u = src_rm.user.get(mb.user.id)
    src_users_login[u.login] = u
    src_users_id[u.id] = u
    
print(src_users_login.keys())

print("Dest Redmine users")
dst_users_login = {}
dst_users_id = {}
for mb in dst_prj.memberships:
    u = dst_rm.user.get(mb.user.id)
    dst_users_login[u.login] = u
    dst_users_id[u.id] = u
    
print(dst_users_login.keys())


Comprobamos si existen todos los miembros del proyecto origen en el proyecto destino

In [ ]:
for k in src_users_login:
    if k not in dst_users_login.keys():
        print("WARNING: EL USUARIO",k,"no es miembro del proyecto destino")

Ahora comprobamos los status de uno y otro lado

In [ ]:
print("Source Redmine statuses")
src_statuses = {}
src_statuses_id = {}
for st in src_rm.issue_status.all():
    src_statuses[st.name] = st
    src_statuses_id[st.id] = st
    
print(src_statuses.keys())

print("Dest Redmine statuses")
dst_statuses = {}
dst_statuses_id = {}
for st in dst_rm.issue_status.all():
    dst_statuses[st.name] = st
    dst_statuses_id[st.id] = st
    
print(dst_statuses.keys())

for k in src_statuses:
    if k not in dst_statuses.keys():
        print("WARNING: EL ESTADO ",k,"no es existe en el servidor destino")


Definiremos cómo se copian las tareas, según exista ya la tarea destino, o según sea una tarea a crear

In [ ]:
cf_normal = ['IssChapter', 'IssTitle', 'Start date', 'WrkloadPct']
cf_users = ['Supervisor', 'Collaborators']
cf_relations = ['Depend']



def calculamos_cfs(srcTsk):
    tmpCfs=[]
    for k in src_cfs.keys():
        #print("\n******",k)
        #print("cf:",src_cfs[k].name)
        if k not in dst_cfs.keys():
            print("WARNING: no existe en el servidor destino el campo",k)
            
        else:
            #print("cfsid:",src_cfs[k].id)
            cv = srcTsk.custom_fields.get(src_cfs[k].id)
            if cv is not None:
                if hasattr(cv,'value'):
                    cv_val = cv.value
                    if cv_val is not None and cv_val is not '':
                        #print("tsk.cv",cv_val)
                        if (k in cf_normal):
                            tmpCfs += [{'id' : src_cfs[k].id, 'value': cv_val}]
                        elif (k in cf_users):
                            #print("cf users",k)
                            if k == 'Supervisor':
                                #print("Supervisor:",cv_val)
                                srcuser = src_users_id[int(cv_val)]
                                dstuser = dst_users_login[srcuser.login]
                                tmpCfs += [{'id' : src_cfs[k].id, 'value': dstuser.id}]
                            elif k == 'Collaborators':
                                #print("Collaborators:",cv_val)
                                colvector = []
                                for col in cv_val:
                                    srcuser = src_users_id[int(col)]
                                    dstuser = dst_users_login[srcuser.login]
                                    colvector += [str(dstuser.id)]
                                
                                tmpCfs += [{'id' : src_cfs[k].id, 'value': colvector}]
                                         
                    
    print("Result:",tmpCfs)
    return tmpCfs

def find_dst_assignee(src_assignee):
    ret = None
    if src_assignee is not None:
        ret = dst_users_login[src_users_id[src_assignee.id].login]

    return ret

def find_dst_status(src_status):
    ret = None
    if src_status is not None:
        ret = dst_statuses[src_status.name]

    return ret

def update_tsk(srcTsk,dstTsk):
    global dst_rm,dst_cfs,src_cfs
    print("\n>>>>>",srcTsk)
    tmpCfs=calculamos_cfs(srcTsk)
    if hasattr(srcTsk,'assigned_to'):
        dest_assignee_id = find_dst_assignee(srcTsk.assigned_to).id
    else:
        dest_assignee_id = None
        

    dst_rm.issue.update(
        dstTsk.id,
        tracker_id=srcTsk.tracker.id,
        description=srcTsk.description+"_pepe",
        status_id=find_dst_status(srcTsk.status).id,
        #priority_id=srcTsk.priority_id,
        assigned_to_id=dest_assignee_id,
        #parent_issue_id=srcTsk.parent_issue_id,
        start_date=srcTsk.start_date,
        due_date=srcTsk.due_date,
        #estimated_hours=srcTsk.estimated_hours,
        custom_fields=tmpCfs,
        done_ratio=srcTsk.done_ratio,
    )
    
def copy_tsk(srcTsk):
    global dst_rm,dst_cfs,src_cfs
    print("\n>>>>>",srcTsk)
    tmpCfs=calculamos_cfs(srcTsk)
    if hasattr(srcTsk,'assigned_to'):
        dest_assignee_id = find_dst_assignee(srcTsk.assigned_to).id
    else:
        dest_assignee_id = None
        
    ret = dst_rm.issue.create(
        project_id=dst_prj.id,
        tracker_id=srcTsk.tracker.id,
        description=srcTsk.description,
        #status_id=srcTsk.status_id,
        #priority_id=srcTsk.priority_id,
        assigned_to_id=dest_assignee_id,
        #parent_issue_id=srcTsk.parent_issue_id,
        start_date=srcTsk.start_date,
        due_date=srcTsk.due_date,
        #estimated_hours=srcTsk.estimated_hours,
        custom_fields=tmpCfs,
        done_ratio=srcTsk.done_ratio
    )
    return ret

Ahora identificaremos, para cada tarea de origen, su tarea destino

In [ ]:
for k in src_tsks.keys():
    if k in dst_tsks.keys():
        print("ya existe")
        update_tsk(src_tsks[k],dst_tsks[k])
        
    else:
        print("hará falta una tarea nueva")
        dst_tsks[k] = copy_tsk(src_tsks[k])
        
    src_tsks[k]['dst'] = dst_tsks[k]

Ahora trataremos las relaciones

In [ ]:
def calculamos_cfs_rel(srcTsk):
    tmpCfs=[]
    for k in src_cfs.keys():
        #print("\n******",k)
        #print("cf:",src_cfs[k].name)
        if k not in dst_cfs.keys():
            print("WARNING: no existe en el servidor destino el campo",k)
            
        else:
            #print("cfsid:",src_cfs[k].id)
            cv = srcTsk.custom_fields.get(src_cfs[k].id)
            if cv is not None:
                if hasattr(cv,'value'):
                    cv_val = cv.value
                    if cv_val is not None and cv_val is not '':
                        if (k in cf_relations):
                            print("cf relations:",k)
                            print("cv_val",cv_val)
                            relvector = []
                            srcrelvector = cv_val.split(',')
                            for rel in srcrelvector:
                                srcrel = src_tsks_id[int(rel)]
                                dstrel = dst_tsks[srcrel.subject]
                                relvector += [dstrel.id]
                                
                            tmpCfs += [{'id' : src_cfs[k].id, 'value': ','.join(map(str, relvector))}]
                            
                    
    print("Result:",tmpCfs)
    return tmpCfs

def update_tsk_rel(srcTsk,dstTsk):
    global dst_rm,dst_cfs,src_cfs
    print("\n>>>>>",srcTsk)
    tmpCfs=calculamos_cfs_rel(srcTsk)

    if hasattr(srcTsk,'parent'):
        print(srcTsk.parent)
        print(srcTsk.parent.id)
        print(src_tsks_id[srcTsk.parent.id].subject)        
        dest_parent_issue_id = src_tsks[src_tsks_id[srcTsk.parent.id].subject]['dst'].id
    else:
        dest_parent_issue_id = None    
    
    dst_rm.issue.update(
        dstTsk.id,
        parent_issue_id=dest_parent_issue_id,
        custom_fields=tmpCfs,
    )
    
    

for k in src_tsks.keys():
    if k in dst_tsks.keys():
        print("ya existe")
        update_tsk_rel(src_tsks[k],dst_tsks[k])

    

In [ ]:
for k in src_tsks.keys():
    if k in dst_tsks.keys():
        print("****",k)
        for r in src_tsks[k].relations:
            if (r.relation_type=='precedes'):
                if (src_tsks[k].id==r.issue_to_id):
                    print(src_tsks[k].id,dict(r))
                    dest_rel_issue = src_tsks[src_tsks_id[r.issue_id].subject]['dst']
                    print(dest_rel_issue)
                    relation = redmine.issue_relation.create(
                        issue_id=12345,
                        issue_to_id=54321,
                        relation_type='precedes',
            ...     delay=5
            ... )
                    dst_tsks[k].relations.add
                    
                
        

In [ ]:
print("hecho")

Necesitaremos ajustar el contador del proyecto